# <img src="https://upload.wikimedia.org/wikipedia/commons/6/60/NISAR_artist_concept.jpg" width=400 align="left"/>
<img src="https://upload.wikimedia.org/wikipedia/commons/9/9b/NISAR_Mission_Logo.png" width=400 align="left"/><br><br><br><br><br>



# Convert ALOS2 Data to GCOV

In preparation for NISAR, this notebook converts ALOS-2 SLC data to NISAR RSLC to NISAR GCOV.


These steps can be run in the command line, details are below.

In [1]:
import os
import sys
from pathlib import Path
import subprocess
import fnmatch
import zipfile
import h5py
from string import Template
from osgeo import gdal
import numpy as np
import math

## Choose your place to save your outputs. 

The home directory should be used (not scratch) unless very large storage space is needed. Files should be moved to the S3 bucket after processing. 

In [2]:
working_dir = Path(os.get_cwd())


## Choose your AOI

New directories will be made to store files for this AOI

In [3]:
aoi = 'waxlakedelta'

aoi_dir = working_dir/aoi
RSLC_dir = aoi_dir / 'RSLC'
ALOS2_dir = aoi_dir / 'ALOS2'
DEM_dir = aoi_dir / 'DEM'
GCOV_dir = aoi_dir / 'GCOV'
TMP_dir = aoi_dir / 'TMP'

Path(aoi_dir).mkdir(parents=True, exist_ok=True)
Path(RSLC_dir).mkdir(parents=True, exist_ok=True)
Path(ALOS2_dir).mkdir(parents=True, exist_ok=True)
Path(DEM_dir).mkdir(parents=True, exist_ok=True)
Path(GCOV_dir).mkdir(parents=True, exist_ok=True)
Path(TMP_dir).mkdir(parents=True, exist_ok=True)



## Copy the ALOS2 URLs into a list

In [4]:
list_of_ALOS_SLCs = ['https://cumulus.asf.alaska.edu/L1.1/A4/0000436588_001001_ALOS2390510580-210816.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000470239_001001_ALOS2427770580-220425.zip']
                        

## Download the ALOS2 files using wget

In [5]:
for file in list_of_ALOS_SLCs:
    print(file)
    filename = ALOS2_dir / file.split('/')[-1]
    if os.path.isfile(filename)==False:
        command = f"wget -P {ALOS2_dir} -q {file}"
        output = subprocess.check_output(command, shell=True)


https://cumulus.asf.alaska.edu/L1.1/A4/0000436588_001001_ALOS2390510580-210816.zip
https://cumulus.asf.alaska.edu/L1.1/A4/0000470239_001001_ALOS2427770580-220425.zip


## Unzip the ALOS files

If you don't want to keep the zip files, you should uncomment the os.remove line

In [11]:
zipfiles = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(ALOS2_dir)
                for f in fnmatch.filter(files,'*.zip')]
for file in zipfiles:
    print(file)
    if os.path.isdir(ALOS2_dir / file[:-4])==False:
        ##check full unzipping
        # if sum(os.path.getsize(ALOS2_dir / file[:-4] / f) for f in os.listdir(ALOS2_dir / file[:-4]) if os.path.isfile(ALOS2_dir / file[:-4] /f)) < 6509714637:
            # with zipfile.ZipFile(file, 'r') as zip_ref:
            #     zip_ref.extractall(ALOS2_dir/file.split('/')[-1][:-4])
            # # os.remove(file)
        with zipfile.ZipFile(file, 'r') as zip_ref:
            zip_ref.extractall(ALOS2_dir/file.split('/')[-1][:-4])


/scratch/alex_eco_test/waxlakedelta/ALOS2/0000436588_001001_ALOS2390510580-210816.zip
/scratch/alex_eco_test/waxlakedelta/ALOS2/0000470239_001001_ALOS2427770580-220425.zip


## Get a list of folders for each ALOS2 file

In [7]:
ALOS2folders = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(ALOS2_dir)
                for f in fnmatch.filter(dirnames,'*')]
ALOS2folders

['/scratch/alex_eco_test/waxlakedelta/ALOS2/0000470239_001001_ALOS2427770580-220425',
 '/scratch/alex_eco_test/waxlakedelta/ALOS2/0000436588_001001_ALOS2390510580-210816']

## Convert ALOS2 SLC to NISAR RSLC format

In [12]:
for ALOS2folder in ALOS2folders[:]:
    ALOS2_id = ALOS2folder.split('/')[-1]
    print('')
    print(ALOS2_id)
    
    ##ALOS2 SLC --> NISAR RSLC
    command = f"conda run -n isce3_src /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i {ALOS2folder} -o {RSLC_dir/ALOS2_id}.h5"
    print('')
    print(command)    
    if os.path.isfile(f"{RSLC_dir/ALOS2_id}.h5")==False: #or os.path.getsize(f"{RSLC_dir/ALOS2_id}.h5")<6000000000:
        try: os.remove(f"{RSLC_dir/ALOS2_id}.h5")
        except:''
        output = subprocess.check_output(command, shell=True)


0000470239_001001_ALOS2427770580-220425

conda run -n isce3_src /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i /scratch/alex_eco_test/waxlakedelta/ALOS2/0000470239_001001_ALOS2427770580-220425 -o /scratch/alex_eco_test/waxlakedelta/RSLC/0000470239_001001_ALOS2427770580-220425.h5

0000436588_001001_ALOS2390510580-210816

conda run -n isce3_src /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i /scratch/alex_eco_test/waxlakedelta/ALOS2/0000436588_001001_ALOS2390510580-210816 -o /scratch/alex_eco_test/waxlakedelta/RSLC/0000436588_001001_ALOS2390510580-210816.h5


In [13]:
ALOS2_RSLCs = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(RSLC_dir)
                for f in fnmatch.filter(files,'*')]
ALOS2_RSLCs

['/scratch/alex_eco_test/waxlakedelta/RSLC/0000436588_001001_ALOS2390510580-210816.h5',
 '/scratch/alex_eco_test/waxlakedelta/RSLC/0000470239_001001_ALOS2427770580-220425.h5']

## Get a DEM for the first ALOS2 file in the stack 

In [14]:

ALOS2_ref = ALOS2_RSLCs[0].split('/')[-1][:-3]
print('')
print(ALOS2_ref)

## Get NISAR DEM
command = f"conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p {RSLC_dir/ALOS2_ref}.h5 -o {DEM_dir/ALOS2_ref}.vrt"
print('')
print(command)    

if os.path.isfile(f"{DEM_dir/ALOS2_ref}.vrt")==False:
    output = subprocess.check_output(command, shell=True)


0000436588_001001_ALOS2390510580-210816

conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p /scratch/alex_eco_test/waxlakedelta/RSLC/0000436588_001001_ALOS2390510580-210816.h5 -o /scratch/alex_eco_test/waxlakedelta/DEM/0000436588_001001_ALOS2390510580-210816.vrt


## Process NISAR RSLC to GCOV

### If you want to change to UTM coordiates, set utm = True and choose the x and y posting. The default is 20m

In [40]:
x_posting = ''
y_posting = ''

EPSG = ''


utm = True
if utm:
    src = gdal.Open(f"{DEM_dir/ALOS2_ref}.vrt")
    ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
    lrx = ulx + (src.RasterXSize * xres)
    lry = uly + (src.RasterYSize * yres)

    x1,y1,x2,y2 = math.floor(ulx),math.floor(uly),math.floor(lrx),math.floor(lry)

    zone = int(np.ceil((ulx + 180)/6))

    if y1>=0:
        EPSG = 32600+zone
    elif y1<0:
        EPSG = 32600+zone

    x_posting = 20
    y_posting = 20
    
print(EPSG)

32615


### Run the GCOV processor on the first image in the stack. This image will be used to determine the bounding box of the remaining images in the stack

In [23]:

## NISAR RSLC --> NISAR GCOV
filein = open('gcov_template.yaml')
template = Template(filein.read())
replacements = {'inputfile':  f"{RSLC_dir/ALOS2_ref}.h5",
                'outputfile':  f"{GCOV_dir/ALOS2_ref}_gcov_%s.h5" %(EPSG),
                'demfile': f"{DEM_dir/ALOS2_ref}.vrt",
                'tmp': f"{TMP_dir}/",
                'epsg': EPSG,
                'xposting': x_posting,
                'yposting': y_posting,
                'top_left_y': '',
                'top_left_x : ''
                }
makeoutput = template.substitute(replacements)
file = open('%s/%s.yaml' %(GCOV_dir,ALOS2_ref),'w')
file.write(makeoutput)
file.close()
filein.close()

command = f"conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py {GCOV_dir/ALOS2_ref}.yaml"
print('')
print(command)
if os.path.isfile(f"{GCOV_dir/ALOS2_ref}_gcov_utm.h5")==False:
    output = subprocess.check_output(command, shell=True)



conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py /scratch/alex_eco_test/waxlakedelta/GCOV/0000436588_001001_ALOS2390510580-210816.yaml


overwriting variable {'LD_LIBRARY_PATH'}
/home/jovyan/isce3/build/packages/nisar/workflows/h5_prep.py:207: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  h5py.File(output_hdf5, 'w', libver='latest', swmr=True) as dst_h5:
/home/jovyan/isce3/build/packages/nisar/workflows/h5_prep.py:489: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  with h5py.File(output_hdf5, 'a', libver='latest', swmr=True) as dst_h5:
/home/jovyan/isce3/python/packages/nisar/workflows/gcov.py:765: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  dset.attrs.create('_FillValue', data=np.nan)
/home/jovyan/isce3/python/packages/nisar/workflows/gcov.py:769: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  dset.attrs.crea

## Get top left coordinates of bottom right coordinates of reference image

In [36]:
f = h5py.File(f"{GCOV_dir/ALOS2_ref}_gcov_%s.h5" %(EPSG), "r") 
a_group_key = list(f.keys())[0]
ds_x = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['xCoordinates'][()]      # returns as a h5py dataset object
ds_y = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['yCoordinates'][()]      # returns as a h5py dataset object

ulx = ds_x[0]
lrx = ds_x[-1]
uly = ds_y[0]
lry = ds_y[-1]
        # 'transform': rasterio.Affine(ds_x[1] - ds_x[0], 0.0, ds_x[0], 0.0, ds_y[1] - ds_y[0], ds_y[0]), 

print('Force top left to be: %s %s' %(ulx,uly))
print('Force top right to be: %s %s' %(lrx,lry))



Force top left to be: 620907.7793358738 3304674.2592810793
Force top right to be: 718447.7793358738 3220074.2592810793


### Process the remaining RSLC files to GCOV using the bounding box set by the reference images

In [38]:
for ALOS2_RSLC in ALOS2_RSLCs[1:2]:
    ALOS2_id = ALOS2_RSLC.split('/')[-1][:-3]
    print('')
    print(ALOS2_id)

  
    ## NISAR RSLC --> NISAR GCOV
    filein = open('gcov_template.yaml')
    template = Template(filein.read())
    replacements = {'inputfile':  f"{RSLC_dir/ALOS2_id}.h5",
                    'outputfile':  f"{GCOV_dir/ALOS2_id}_gcov_%s.h5" %(EPSG),
                    'demfile': f"{DEM_dir/ALOS2_ref}.vrt",
                    'tmp': f"{TMP_dir}/",
                    'epsg': EPSG,
                    'xposting': x_posting,
                    'yposting': y_posting,
                    'top_left_y': uly,
                    'top_left_x':ulx,
                    'bottom_right_y':lry,
                    'bottom_right_x': lrx
                    }
    makeoutput = template.substitute(replacements)
    file = open('%s/%s.yaml' %(GCOV_dir,ALOS2_id),'w')
    file.write(makeoutput)
    file.close()
    filein.close()

    command = f"conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py {GCOV_dir/ALOS2_id}.yaml"
    print('')
    print(command)
    if os.path.isfile(f"{GCOV_dir/ALOS2_id}_gcov_utm.h5")==False:
        output = subprocess.check_output(command, shell=True)



0000436588_001001_ALOS2390510580-210816

conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py /scratch/alex_eco_test/waxlakedelta/GCOV/0000436588_001001_ALOS2390510580-210816.yaml


overwriting variable {'LD_LIBRARY_PATH'}
/home/jovyan/isce3/build/packages/nisar/workflows/h5_prep.py:207: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  h5py.File(output_hdf5, 'w', libver='latest', swmr=True) as dst_h5:
/home/jovyan/isce3/build/packages/nisar/workflows/h5_prep.py:489: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  with h5py.File(output_hdf5, 'a', libver='latest', swmr=True) as dst_h5:
/home/jovyan/isce3/python/packages/nisar/workflows/gcov.py:765: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  dset.attrs.create('_FillValue', data=np.nan)
/home/jovyan/isce3/python/packages/nisar/workflows/gcov.py:769: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  dset.attrs.crea


0000470239_001001_ALOS2427770580-220425

conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py /scratch/alex_eco_test/waxlakedelta/GCOV/0000470239_001001_ALOS2427770580-220425.yaml


overwriting variable {'LD_LIBRARY_PATH'}
/home/jovyan/isce3/build/packages/nisar/workflows/h5_prep.py:207: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  h5py.File(output_hdf5, 'w', libver='latest', swmr=True) as dst_h5:
/home/jovyan/isce3/build/packages/nisar/workflows/h5_prep.py:489: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  with h5py.File(output_hdf5, 'a', libver='latest', swmr=True) as dst_h5:
/home/jovyan/isce3/python/packages/nisar/workflows/gcov.py:765: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  dset.attrs.create('_FillValue', data=np.nan)
/home/jovyan/isce3/python/packages/nisar/workflows/gcov.py:769: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  dset.attrs.crea

## See final list of NISAR GCOV data files

In [41]:
NISAR_GCOVs = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(GCOV_dir)
                for f in fnmatch.filter(files,'*.h5')]
NISAR_GCOVs

['/scratch/alex_eco_test/waxlakedelta/GCOV/0000436588_001001_ALOS2390510580-210816_gcov.h5',
 '/scratch/alex_eco_test/waxlakedelta/GCOV/0000436588_001001_ALOS2390510580-210816_gcov_32615.h5',
 '/scratch/alex_eco_test/waxlakedelta/GCOV/0000470239_001001_ALOS2427770580-220425_gcov.h5',
 '/scratch/alex_eco_test/waxlakedelta/GCOV/0000470239_001001_ALOS2427770580-220425_gcov_32615.h5']

In [ ]:
## Command line option

######## ALOS2 SLC to NISAR RSLC
# for f in /scratch/alex_eco_test/ALOS2/southfork/*/; 
# do python /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i $f -o "${f%.*}".h5; 
# done

######## Get NISAR DEM
# for f in /scratch/alex_eco_test/ALOS2/southfork/rslc/*; 
# do python /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p $f -o "${f%.*}".vrt; 
# done

######## NISAR RSLC to NISAR GCOV
# for f in /scratch/alex_eco_test/ALOS2/southfork/rslc/*.yaml; 
# do python /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py $f; 
# done